In [5]:
#pip install yake

     ---------------------------------------- 60.2/60.2 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import json
import datasets
from datasets import load_dataset

In [13]:
# get the dataset only for keyphrase generation
dataset = load_dataset("midas/duc2001", "generation")

print("Samples for Keyphrase Generation")

# sample from the test split
print("Sample from test data split")
test_sample = dataset["test"][0]
print("Fields in the sample: ", [key for key in test_sample.keys()])
print("Tokenized Document: ", test_sample["document"])
print("Extractive/present Keyphrases: ", test_sample["extractive_keyphrases"])
print("Abstractive/absent Keyphrases: ", test_sample["abstractive_keyphrases"])
print("\n-----------\n")

Found cached dataset duc2001 (C:/Users/Alicia/.cache/huggingface/datasets/midas___duc2001/generation/0.0.1/7888b46165d8a58f49f00e28410b46b1f22fabfd72a9e89f3e80a4e2d27e4a9b)


  0%|          | 0/1 [00:00<?, ?it/s]

Samples for Keyphrase Generation
Sample from test data split
Fields in the sample:  ['id', 'document', 'extractive_keyphrases', 'abstractive_keyphrases']
Tokenized Document:  ['Here', ',', 'at', 'a', 'glance', ',', 'are', 'developments', 'today', 'involving', 'the', 'crash', 'of', 'Pan', 'American', 'World', 'Airways', 'Flight', '103', 'Wednesday', 'night', 'in', 'Lockerbie', ',', 'Scotland', ',', 'that', 'killed', 'all', '259', 'people', 'aboard', 'and', 'more', 'than', '20', 'people', 'on', 'the', 'ground', ':']
Extractive/present Keyphrases:  ['pan american world airways flight 103', 'crash', 'lockerbie']
Abstractive/absent Keyphrases:  ['terrorist threats', 'widespread wreckage', 'radical palestinian faction', 'terrorist bombing', 'bomb threat', 'sabotage']

-----------



In [14]:
def drop_column(dataset):
    del dataset['abstractive_keyphrases']
    return dataset

# Para eliminar la columna que no necesitamos
dataset = dataset.map(drop_column)

Loading cached processed dataset at C:\Users\Alicia\.cache\huggingface\datasets\midas___duc2001\generation\0.0.1\7888b46165d8a58f49f00e28410b46b1f22fabfd72a9e89f3e80a4e2d27e4a9b\cache-16ebf8679ee254b4.arrow


Hasta aquí tenemos un dataset con las 3 columnas de interés:

In [15]:
dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'document', 'extractive_keyphrases'],
        num_rows: 308
    })
})

#######################################################################################################################################################################################

In [8]:
import yake

In [23]:
language = "en"
max_ngram_size = 2
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)


In [24]:
keywords = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in custom_kw_extractor.extract_keywords(joined_text):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append((keyword)) # guardar información en una tupla
    keywords.append(keywords_row) # guardar información de palabras clave para cada fila

In [53]:
keywords[7] #prueba  # DEBERIAMOS ELIMINAR LAS Q SIN QUERER SALE Q SON DE LONG=3 PORQ SON CASOS RAROS COMO AMARICAN CORP. y AFRICA LTD.  etc.

['South Africa',
 'Fair Trading',
 'Consolidated Gold',
 'Fields PLC',
 'Anglo American',
 'American Corp.',
 'Selling Organization',
 'Africa Ltd.',
 'Consolidated Mines',
 'Beers Consolidated',
 'Gold Fields',
 'Central Selling',
 'Luxembourg-based Minorco',
 'diamond trading',
 'London-based diamond',
 'Beers diamond',
 'Beers',
 'Mergers Commission',
 'South',
 'diamond organization']

A continuación sacamos las categorias gramaticales de cada keyword/keyphrase:

In [26]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag

categorias = []

for keyword_list in keywords:
    sublist_categorias = []
    for keyword in keyword_list:
        palabras = word_tokenize(keyword)
        etiquetas = pos_tag(palabras)
        sublist_categorias.append(etiquetas)
    categorias.append(sublist_categorias)

print(categorias)

[[[('Airways', 'NNS'), ('Flight', 'VBD')], [('Pan', 'NNS'), ('American', 'VBP')], [('American', 'NNP'), ('World', 'NNP')], [('World', 'NNP'), ('Airways', 'NNP')], [('Wednesday', 'NNP'), ('night', 'NN')], [('people', 'NNS'), ('aboard', 'VBP')], [('developments', 'NNS'), ('today', 'NN')], [('today', 'NN'), ('involving', 'VBG')], [('Scotland', 'NN')], [('Flight', 'NN')], [('Wednesday', 'NNP')], [('Lockerbie', 'NN')], [('people', 'NNS')], [('Pan', 'NN')], [('American', 'JJ')], [('World', 'NN')], [('Airways', 'NNS')], [('glance', 'NN')], [('ground', 'NN')], [('developments', 'NNS')]], [[('Weather', 'NNP'), ('Service', 'NNP')], [('weather', 'NN'), ('conditions', 'NNS')], [('College', 'NNP'), ('Station', 'NNP')], [('Gulf', 'NNP'), ('Coast', 'NNP')], [('Laureen', 'NNP'), ('Chernow', 'NNP')], [('Cherokee', 'NNP'), ('County', 'NNP')], [('National', 'NNP'), ('Weather', 'NNP')], [('West', 'NNP'), ('Texas', 'NNP')], [('East', 'NNP'), ('Texas', 'NNP')], [('Alley', 'NNP'), ('states', 'NNS')], [('Torn

In [27]:
print(len(categorias))
print(categorias[0])

308
[[('Airways', 'NNS'), ('Flight', 'VBD')], [('Pan', 'NNS'), ('American', 'VBP')], [('American', 'NNP'), ('World', 'NNP')], [('World', 'NNP'), ('Airways', 'NNP')], [('Wednesday', 'NNP'), ('night', 'NN')], [('people', 'NNS'), ('aboard', 'VBP')], [('developments', 'NNS'), ('today', 'NN')], [('today', 'NN'), ('involving', 'VBG')], [('Scotland', 'NN')], [('Flight', 'NN')], [('Wednesday', 'NNP')], [('Lockerbie', 'NN')], [('people', 'NNS')], [('Pan', 'NN')], [('American', 'JJ')], [('World', 'NN')], [('Airways', 'NNS')], [('glance', 'NN')], [('ground', 'NN')], [('developments', 'NNS')]]


In [28]:
print(len(categorias[0][0])) #  Airways Flight
print(len(categorias[0][1])) # Pan American 

print(len(categorias[0])) # las 20 keywords/keyphrases del doc 0

2
2
20


In [29]:
longitudes = []

for sublist in categorias:
    for subsublist in sublist:
        longitudes.append(len(subsublist))

print(longitudes)

[2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 

In [30]:
from collections import Counter

count_longitudes = Counter(longitudes)
print(dict(count_longitudes)) # todas tienen long 1 ,2 o 3

{2: 5123, 1: 989, 3: 21}


In [42]:
posiciones_tres = [i for i in range(len(longitudes)) if longitudes[i] == 3]
posiciones_tres

[41,
 145,
 147,
 291,
 381,
 464,
 511,
 559,
 760,
 983,
 1346,
 1362,
 1364,
 1374,
 1678,
 2116,
 2128,
 2526,
 2818,
 3714,
 3726]

In [52]:

categorias[7] # vemos que las que nos sale que tienen long 3 es porque van acompañandas de un signo. Ej- American Corp.   Africa Ltd.  

[[('South', 'NNP'), ('Africa', 'NNP')],
 [('Fair', 'CD'), ('Trading', 'NN')],
 [('Consolidated', 'NNP'), ('Gold', 'NNP')],
 [('Fields', 'NNS'), ('PLC', 'NNP')],
 [('Anglo', 'JJ'), ('American', 'JJ')],
 [('American', 'NNP'), ('Corp', 'NNP'), ('.', '.')],
 [('Selling', 'VBG'), ('Organization', 'NN')],
 [('Africa', 'NNP'), ('Ltd', 'NNP'), ('.', '.')],
 [('Consolidated', 'JJ'), ('Mines', 'NNS')],
 [('Beers', 'NNS'), ('Consolidated', 'VBD')],
 [('Gold', 'NN'), ('Fields', 'NNS')],
 [('Central', 'JJ'), ('Selling', 'NN')],
 [('Luxembourg-based', 'JJ'), ('Minorco', 'NNP')],
 [('diamond', 'NN'), ('trading', 'NN')],
 [('London-based', 'JJ'), ('diamond', 'NN')],
 [('Beers', 'NNS'), ('diamond', 'VBP')],
 [('Beers', 'NNS')],
 [('Mergers', 'NNS'), ('Commission', 'NNP')],
 [('South', 'NNP')],
 [('diamond', 'NN'), ('organization', 'NN')]]

In [57]:
categorias[14] # más de lo mismo: occured Aug.

[[('Richter', 'NNP'), ('scale', 'NN')],
 [('Soviet', 'NNP'), ('Union', 'NNP')],
 [('Geological', 'NNP'), ('Survey', 'NNP')],
 [('death', 'NN'), ('toll', 'NN')],
 [('earthquake', 'NN'), ('measuring', 'NN')],
 [('earthquake', 'NN')],
 [('Survey', 'NNP'), ('reported', 'VBD')],
 [('Chinese', 'JJ'), ('earthquake', 'NN')],
 [('Armenian', 'JJ'), ('earthquake', 'NN')],
 [('Soviet', 'JJ'), ('central', 'JJ')],
 [('Richter', 'NN')],
 [('occurred', 'VBD'), ('Aug', 'NNP'), ('.', '.')],
 [('earthquake', 'NN'), ('occurred', 'VBD')],
 [('Armenia', 'NNP'), ('pushed', 'VBD')],
 [('significant', 'JJ'), ('earthquakes', 'NNS')],
 [('earthquake', 'NN'), ('death', 'NN')],
 [('occurred', 'VBN'), ('March', 'NNP')],
 [('central', 'JJ'), ('Asia', 'NNP')],
 [('toll', 'NN')],
 [('toll', 'NN'), ('worldwide', 'NN')]]

A partir de aquí podriamos mirar de descartar los [i][j] que tienen len =1, y los que tienen 2 mirar de si su combinación es una de las de abajo (las del paper aquel pág 4)

In [58]:
# en esta se eliminan las que tienen long 1
keyphrases_pos = []

for sublista in categorias:
    nueva_sublista = []
    for subsublista in sublista:
        if len(subsublista) > 1:
            nueva_sublista.append(subsublista)
    if nueva_sublista:
        keyphrases_pos.append(nueva_sublista)

print(keyphrases_pos)

[[[('Airways', 'NNS'), ('Flight', 'VBD')], [('Pan', 'NNS'), ('American', 'VBP')], [('American', 'NNP'), ('World', 'NNP')], [('World', 'NNP'), ('Airways', 'NNP')], [('Wednesday', 'NNP'), ('night', 'NN')], [('people', 'NNS'), ('aboard', 'VBP')], [('developments', 'NNS'), ('today', 'NN')], [('today', 'NN'), ('involving', 'VBG')]], [[('Weather', 'NNP'), ('Service', 'NNP')], [('weather', 'NN'), ('conditions', 'NNS')], [('College', 'NNP'), ('Station', 'NNP')], [('Gulf', 'NNP'), ('Coast', 'NNP')], [('Laureen', 'NNP'), ('Chernow', 'NNP')], [('Cherokee', 'NNP'), ('County', 'NNP')], [('National', 'NNP'), ('Weather', 'NNP')], [('West', 'NNP'), ('Texas', 'NNP')], [('East', 'NNP'), ('Texas', 'NNP')], [('Alley', 'NNP'), ('states', 'NNS')], [('Tornado', 'NNP'), ('Alley', 'NNP')], [('central', 'JJ'), ('Louisiana', 'NNP')], [('early', 'RB'), ('March', 'NNP')], [('small', 'JJ'), ('West', 'NNP')], [('Texas', 'NNP'), ('town', 'NN')], [('Buddy', 'NNP'), ('McIntyre', 'NNP')], [('people', 'NNS'), ('died', 'V

In [59]:
len(keyphrases_pos) # Seguimos teniendo 308 sublistas (una para cada doc)

308

In [60]:
print(len(keyphrases_pos[0])) # antes contenia 20 keywords/keyphrases y ahora eliminando las de long 1 se han quedado en 8

8


Quitamos las que tienen una combinación diferente a las que nos interesan:

In [61]:
valid_combinations = [('JJ', 'NN'), ('NN', 'NN'), ('NNS', 'NN'), ('NNP', 'NN'), ('NNPS', 'NN')]

for i in range(len(keyphrases_pos)):
    for j in range(len(keyphrases_pos[i])):
        if isinstance(keyphrases_pos[i][j], list):
            for k in range(len(keyphrases_pos[i][j]) - 1):
                if (keyphrases_pos[i][j][k][1], keyphrases_pos[i][j][k+1][1]) not in valid_combinations:
                    keyphrases_pos[i][j] = []
                    break
        else:
            if (keyphrases_pos[i][j][1],) not in valid_combinations:
                keyphrases_pos[i][j] = []

# Eliminar las sublistas vacías de prueba
#keyphrases_pos = [x for x in keyphrases_pos if x != []] NO VA! MIRAR OTRA MANERA SI ME LLEGAN A ESTORBAR

print(keyphrases_pos)


[[[], [], [], [], [('Wednesday', 'NNP'), ('night', 'NN')], [], [('developments', 'NNS'), ('today', 'NN')], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [('Texas', 'NNP'), ('town', 'NN')], [], [], [], [('property', 'NN'), ('damage', 'NN')], []], [[], [], [], [], [], [('force', 'NN'), ('spokeswoman', 'NN')], [('German', 'JJ'), ('police', 'NN')], [], [], [], [], [('air', 'NN'), ('today', 'NN')], [('Tactical', 'JJ'), ('Fighter', 'NN')], [('force', 'NN'), ('base', 'NN')], [('tactical', 'JJ'), ('air', 'NN')], [('training', 'NN'), ('mission', 'NN')]], [[], [], [], [], [('fire', 'NN'), ('management', 'NN')], [], [], [], [], [('Western', 'JJ'), ('fire', 'NN')], [('fire', 'NN'), ('season', 'NN')], [], [], [], [], [], [], [], [('drought-primed', 'JJ'), ('Yellowstone', 'NN')]], [[], [], [], [], [], [], [], [('Sunday', 'NNP'), ('school', 'NN')], [], [], [], [], [], [], [('Ozark', 'NNP'), ('tradition', 'NN')], [], [], [('arson', 'NN'), ('fire', 'NN')]], [[], [], [('Day', 'NNP'), ('w

In [62]:
len(keyphrases_pos)

308

############################################################################################################################################################

Ahora calcularemos la precisión i la similitud de Jaccard para cada texto, para ello necesitamos saber cuántas extractive_keyphrases tiene cada uno y cuántas de estas coinciden con las keyphrases obtenidas con KeyBERT y el filtraje hecho con POS

Juntamos las extractive keyphrases en una lista, ya que están separadas en el dataset

In [65]:
# Crear una lista de sublistas
extractive_keyphrases_list = []

for element in dataset['test']:
    # Acceder a la lista de extractive_keyphrases para cada elemento
    extractive_keyphrases = element['extractive_keyphrases']
    
    # Añadir la lista de extractive_keyphrases a la lista de sublistas
    extractive_keyphrases_list.append(extractive_keyphrases)
    
print(len(extractive_keyphrases_list))
print(extractive_keyphrases_list)

308
[['pan american world airways flight 103', 'crash', 'lockerbie'], ['tornado season', 'spring thunderstorms', 'tornadoes', 'texas', 'property damage', 'tornado warning', 'tornado watches', 'disaster research'], ['crashes', 'pilots', 'bodenheim', 'training mission', 'aircraft'], ['forest fire policy', 'western fire season', 'national forests', 'fire management plans', 'natural fires', 'recommended halt', 'panel'], ['forest fires', 'criminal investigator', 'investigations', 'passing truck', 'missouri', 'arsonist', 'arson problem'], ['welfare reform', 'president reagan', 'welfare legislation', 'work requirement', 'welfare programs', 'unemployment', 'americans'], ['crash', 'training flight', 'crew members', 'pilot'], ['de beers diamond organization', 'fair trading', 'south africa', 'world diamond trading', 'defensive move', 'central selling organization', 'investigation'], ['deadening drought', 'annual agriculture department survey', 'fall corn harvest', 'higher retail food prices', 'pr

In [66]:
# guardar el numero de extractive keyphrases que hay para cada doc (esto es necesario para luego calcular la precisión)

n_extractive_keyphrases = []
for sublista in extractive_keyphrases_list:
    num_elementos = len(sublista)
    n_extractive_keyphrases.append(num_elementos)

print(n_extractive_keyphrases)

[3, 8, 5, 7, 7, 7, 4, 7, 9, 6, 7, 8, 6, 5, 7, 10, 7, 7, 5, 5, 8, 7, 8, 6, 7, 8, 6, 6, 2, 5, 7, 8, 8, 8, 8, 8, 8, 6, 7, 9, 8, 9, 8, 5, 7, 8, 5, 8, 7, 8, 12, 7, 5, 10, 10, 6, 10, 7, 4, 10, 7, 8, 8, 7, 10, 4, 3, 10, 7, 9, 6, 6, 5, 6, 6, 6, 7, 8, 6, 2, 9, 7, 6, 6, 7, 7, 3, 6, 6, 8, 6, 6, 4, 7, 8, 6, 6, 9, 7, 6, 7, 5, 6, 8, 5, 6, 4, 6, 7, 10, 6, 8, 7, 7, 9, 8, 6, 8, 3, 10, 7, 8, 4, 7, 12, 9, 9, 7, 7, 5, 10, 9, 6, 10, 7, 11, 4, 6, 11, 8, 13, 8, 10, 7, 8, 7, 9, 8, 11, 7, 7, 6, 6, 5, 5, 13, 6, 8, 4, 9, 8, 6, 8, 8, 12, 10, 9, 6, 7, 7, 10, 6, 4, 7, 8, 6, 11, 8, 8, 5, 5, 7, 9, 7, 7, 6, 8, 6, 9, 5, 8, 11, 8, 6, 9, 11, 10, 7, 7, 7, 7, 9, 7, 7, 8, 11, 7, 7, 7, 6, 8, 6, 5, 7, 7, 11, 9, 9, 8, 7, 7, 7, 8, 9, 8, 5, 6, 7, 6, 10, 4, 11, 7, 6, 8, 9, 4, 7, 5, 8, 11, 6, 6, 9, 5, 10, 5, 7, 8, 6, 6, 5, 7, 6, 6, 8, 6, 7, 9, 10, 7, 4, 5, 4, 10, 3, 6, 6, 5, 9, 7, 7, 7, 7, 7, 7, 7, 7, 8, 6, 6, 10, 6, 6, 10, 6, 6, 12, 7, 8, 8, 8, 7, 6, 5, 5, 7, 5, 9, 5, 7, 5, 6, 5, 6, 7, 7, 6]


Ahora vamos a tener que juntar las keyphrases de la lista keyphrases_pos (tmbn de long = 308) para luego ver para cada sublista de esta lista los matches con cada sublista de las extractive_keyphrases_list

In [67]:
keyphrases_pos_list = []
for i in range(len(keyphrases_pos)):
    temp = []
    for j in range(len(keyphrases_pos[i])):
        for k in range(len(keyphrases_pos[i][j])-1):
            if (keyphrases_pos[i][j][k][1], keyphrases_pos[i][j][k+1][1]) in valid_combinations:
                temp.append(keyphrases_pos[i][j][k][0] + ' ' + keyphrases_pos[i][j][k+1][0])
    keyphrases_pos_list.append(temp)

print(len(keyphrases_pos_list))
print(keyphrases_pos_list)

308
[['Wednesday night', 'developments today'], ['Texas town', 'property damage'], ['force spokeswoman', 'German police', 'air today', 'Tactical Fighter', 'force base', 'tactical air', 'training mission'], ['fire management', 'Western fire', 'fire season', 'drought-primed Yellowstone'], ['Sunday school', 'Ozark tradition', 'arson fire'], ['Day weekend', 'welfare reform', 'work requirement', 'vacation ranch', 'weekend radio', 'radio address', 'welfare legislation'], ['Force Base', 'base spokesman'], ['Central Selling', 'diamond trading', 'London-based diamond', 'diamond organization'], ['crop production'], ['sprinter Angella', 'Star newspaper', 'Olympic sprinter', 'Olympic gold', 'gold medal'], ['jet shut', 'crew shut', 'Fatal Error', 'flight crew'], ['partial eclipse', 'total eclipse', 'Mindanao city', 'Philippine island', 'wide area'], ['oil spill', 'crude oil'], ['Memorial Hospital'], ['Richter scale', 'death toll', 'earthquake measuring', 'Chinese earthquake', 'Armenian earthquake',

In [68]:
#ir viendo los matches
print(extractive_keyphrases_list[1])
print(keyphrases_pos_list[1])

['tornado season', 'spring thunderstorms', 'tornadoes', 'texas', 'property damage', 'tornado warning', 'tornado watches', 'disaster research']
['Texas town', 'property damage']


Miramos los matches para cada doc

In [69]:
n_matches = []
for sub1, sub2 in zip(keyphrases_pos_list, extractive_keyphrases_list):
    contador = 0
    for elem in sub1:
        if elem in sub2:
            contador += 1
    n_matches.append(contador)

print(len(n_matches))
print(n_matches)


308
[0, 1, 1, 0, 0, 3, 0, 0, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 3, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0, 3, 3, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 1, 0, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 3, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 1, 0, 2, 2, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 3, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0]


Para calcular la precisión obtenida en cada doc, tenemos que calcular: n keyphrases obtenidas con keybert que hacen match / las extractive keyphrases que hay para cada doc   (n_matches/n_extractive_keyphrases)

In [70]:
precision = []

for x, y in zip(n_matches, n_extractive_keyphrases):
    precision.append(x / y)

print(precision)

[0.0, 0.125, 0.2, 0.0, 0.0, 0.42857142857142855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0.3333333333333333, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0, 0.125, 0.0, 0.0, 0.125, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.125, 0.0, 0.125, 0.0, 0.125, 0.0, 0.14285714285714285, 0.1111111111111111, 0.125, 0.0, 0.375, 0.0, 0.14285714285714285, 0.125, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.14285714285714285, 0.25, 0.0, 0.0, 0.125, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.5, 0.0, 0.0, 0.16666666666666666, 0.0, 0.16666666666666666, 0.14285714285714285, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.125, 0.0, 0.0, 0.2222222222222222, 0.14285714285714285, 0.0, 0.14285714285714285, 0.6, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.16666666666666666, 0.125, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.2222222222222222, 0.0, 0.

media global sencilla:

In [71]:
precision_mean = sum(precision)/len(precision)
print(precision_mean) # con top_n = 20 y nr_candidates=20 la media sale 0.13596246826766314 , si lo cambiamos a 30 baja MUY SIGNIF a 0.013783846872082166

0.07285228299838686


############################################################################################################################################################

Para utilizar la similitud de Jaccard en la evaluación de la precisión en la extracción de palabras clave, se pueden seguir los siguientes pasos:

Obtener el conjunto de palabras clave extraídas y el conjunto de palabras clave correctas.

Calcular la intersección de ambos conjuntos, es decir, el número de palabras clave que son comunes a ambos conjuntos. (n_matches)

Calcular la unión de ambos conjuntos, es decir, el número total de palabras clave distintas en ambos conjuntos.(n_extractive_keyphrases - n_matches)

Calcular la similitud de Jaccard dividiendo el número de palabras clave en la intersección por el número de palabras clave en la unión. (n_matches / [n_extractive_keyphrases - n_matches] )

In [72]:
jaccard = []

for x, y in zip(n_matches, n_extractive_keyphrases):
    jaccard.append(x /(y - x))

print(jaccard)

[0.0, 0.14285714285714285, 0.25, 0.0, 0.0, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6, 0.5, 0.0, 0.0, 0.1111111111111111, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.14285714285714285, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.14285714285714285, 0.0, 0.14285714285714285, 0.0, 0.14285714285714285, 0.0, 0.16666666666666666, 0.125, 0.14285714285714285, 0.0, 0.6, 0.0, 0.16666666666666666, 0.14285714285714285, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.16666666666666666, 0.3333333333333333, 0.0, 0.0, 0.14285714285714285, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 1.0, 0.0, 0.0, 0.2, 0.0, 0.2, 0.16666666666666666, 0.0, 0.5, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.2857142857142857, 0.16666666666666666, 0.0, 0.16666666666666666, 1.5, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.2, 0.14285714285714285, 0.0,

media global sencilla

In [73]:
jaccard_mean = sum(jaccard)/len(jaccard)
print(jaccard_mean)

0.1034967017109874


El resultado de la similitud de Jaccard es un valor entre 0 y 1, donde 0 indica que no hay similitud entre los conjuntos de palabras clave y 1 indica que son idénticos. Un valor cercano a 1 indica una alta precisión en la extracción de palabras clave.